In [ ]:

import json
from pyld import jsonld
import cmipld

In [ ]:
gr = json.load(open('experiment/graph.jsonld', 'r'))
gr['@context']

In [ ]:
# Iterate over all context keys in gr['@context']
context_results = {}
for ctx_key, ctx_val in gr['@context'].items():
    # If the context value is a dict, skip (not a URL)
    if isinstance(ctx_val, dict):
        continue
    # Otherwise, treat as a URL and load
    try:
        loaded = cmipld.loader.loader(ctx_val)
        context_results[ctx_key] = loaded.get(ctx_key)
    except Exception as e:
        context_results[ctx_key] = f"Error: {e}"

context_results

In [ ]:
jsonld.compact(gr,gr)

In [ ]:
jsonld.expand(gr)[0]

In [ ]:
cmipld.loader.loader('https://wcrp-cmip.github.io/WCRP-universe/activity/_context_')

In [ ]:
jsonld.to_rdf(gr)

In [ ]:
from pyld import jsonld

rdf = jsonld.to_rdf(gr)  # returns a dict of graphs

# Collect all IRIs (@id values) from subject, predicate, and object
all_ids = set()

for graph_name, triples in rdf.items():
    for triple in triples:
        for key in ['subject', 'predicate', 'object']:
            term = triple[key]
            if term['type'] == 'IRI':
                all_ids.add(term['value'])

# Print them sorted
for iri in sorted(all_ids):
    print(iri)



In [ ]:
# jsonld.to_rdf(data_list, {'format': 'application/nquads'})

In [ ]:
from pyld import jsonld

# Your JSON-LD data
# gr = ...

# Structured RDF dataset (dict form)
rdf_dataset = jsonld.to_rdf(jsonld.expand(gr))

# Collect linked IRIs (i.e., object IRIs)
linked_ids = set()

for graph_name, triples in rdf_dataset.items():
    for triple in triples:
        obj = triple['object']
        if obj['type'] == 'IRI':
            linked_ids.add(obj['value'])

# Convert to sorted list
linked_ids = sorted(linked_ids)

# Output
for iri in linked_ids:
    print(iri)


In [ ]:
gr['@context']

In [ ]:
jsonld.compact(gr, gr)['@graph'][0]


In [ ]:
jsonld.compact(gr, gr)['@context']

In [ ]:
import cmipld 


In [ ]:
cmipld.jsonld

In [ ]:
cmipld.locations.direct

In [ ]:
from cmipld.utils.git import get_path_url, get_repo_url, get_relative_path, url2io
from cmipld import prefix_url
import cmipld

In [22]:
repo = cmipld.utils.git.get_repo_url()

prefix = cmipld.reverse_direct[repo]

repo,prefix

('https://github.com/wcrp-cmip/CMIP7-CVs/', 'cmip7')

In [ ]:

from cmipld.utils.server_tools.offline_patched import LD_server
import os 

loc = os.getcwd()
repos = [(cmipld.mapping[prefix], prefix)]
server = LD_server(offline=repos, use_ssl=False)
url = server.start_server(port=8081)

TypeError: 'Loader' object is not callable

In [19]:
os.getcwd()

'/Users/daniel.ellis/WIPwork/CMIP7_CVs/src-data'

In [20]:
from pyld import jsonld
jsonld.to_rdf('cmip7:experiment/graph.jsonld')

{'@default': [{'subject': {'type': 'IRI',
    'value': 'https://wcrp-cmip.github.io/CMIP7-CVs/experiment/1pctco2'},
   'predicate': {'type': 'IRI',
    'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
   'object': {'type': 'IRI', 'value': 'wcrp:experiment'}},
  {'subject': {'type': 'IRI',
    'value': 'https://wcrp-cmip.github.io/CMIP7-CVs/experiment/1pctco2'},
   'predicate': {'type': 'IRI',
    'value': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
   'object': {'type': 'IRI',
    'value': 'https://wcrp-cmip.github.io/CMIP7-CVs/'}},
  {'subject': {'type': 'IRI',
    'value': 'https://wcrp-cmip.github.io/CMIP7-CVs/experiment/1pctco2'},
   'predicate': {'type': 'IRI',
    'value': 'https://wcrp-cmip.github.io/CMIP7-CVs/experiment/activity'},
   'object': {'type': 'IRI',
    'value': 'https://wcrp-cmip.github.io/WCRP-universe/activity/cmip'}},
  {'subject': {'type': 'IRI',
    'value': 'https://wcrp-cmip.github.io/CMIP7-CVs/experiment/1pctco2'},
   'predicate': {'type'